In [1]:
import os
from bs4 import BeautifulSoup
import requests
import time
from datetime import datetime
import json
from zipfile import ZipFile
import pandas as pd
import shutil
import zipfile
import random

### Define PATH constant

In [2]:
BASE_PATH = "craw"
HTML_PATH = BASE_PATH + "/html"
USER_PATH = BASE_PATH + "/users"

# 1. Get Clubs IDs

In [3]:
def get_number(string):
    return int(string.strip().replace(",", ""))


clubs_id = set()
possibles_users = 0
page = 1

while True:
    print(f"\r{page}", end="")

    time.sleep(3)  # Wait 3 seconds per page
    data = requests.get(f"https://myanimelist.net/clubs.php?p={page}")
    soup = BeautifulSoup(data.text, "html.parser")
    rows = soup.find_all("tr", {"class": "table-data"})
    for row in rows:
        members = get_number(row.find("td", {"class": "ac"}).text)
        club_id = get_number(
            row.find("a", {"class": "fw-b"}).get("href").split("=")[-1]
        )
        if (
            club_id not in clubs_id and members > 30
        ):  # Only save groups with more than 30 members
            possibles_users += members
            clubs_id.add(club_id)

    page += 1
    # if possibles_users > 1000000:  # Threshold to stop
    if possibles_users > 10:  # Threshold to stop
        break

with open(f"{BASE_PATH}/clubs.txt", "w") as file:
    for club in clubs_id:
        file.write(f"{club}\n")

1

# 2. Get usernames in every clubs

In [4]:
if not os.path.exists(f"{BASE_PATH}/users_list.txt"):
    with open(f"{BASE_PATH}/users_list.txt", "w", encoding="UTF-8") as file:
        pass
    
if not os.path.exists(f"{BASE_PATH}/_revised_clubs.txt"):
    with open(f"{BASE_PATH}/_revised_clubs.txt", "w", encoding="UTF-8") as file:
        pass

In [5]:
with open(f"{BASE_PATH}/clubs.txt") as file:
    clubs_id = [x.strip() for x in file.readlines()]

with open(f"{BASE_PATH}/users_list.txt", encoding="UTF-8") as file:
    users = set([x.strip() for x in file.readlines()])

with open(f"{BASE_PATH}/_revised_clubs.txt", encoding="UTF-8") as file:
    revised_clubs = set([int(x.strip()) for x in file.readlines()])

len(users), len(revised_clubs), len(clubs_id)

(0, 0, 41)

In [7]:
#%%notify -m "get username finish"

for i, club_id in enumerate(clubs_id):
    if club_id in revised_clubs:
        continue

    # page = 1
    # while True:
    print(f"\r{i+1}/{len(clubs_id)} --> {str(page).zfill(2)}", end="")
    # link = f"https://api.jikan.moe/v3/club/{club_id}/members/{page}"
    link = f"https://api.jikan.moe/v4/clubs/{club_id}/members"

    try:
        time.sleep(4.2)
        data = requests.get(link)
    except KeyboardInterrupt:
        raise KeyboardInterrupt()
    except:  # Other exception wait 2 min and try again
        time.sleep(120)
        continue

    if data.status_code != 200:
        break

    with open(f"{BASE_PATH}/users_list.txt", "a", encoding="UTF-8") as file:
        for user in map(lambda x: x["username"], json.loads(data.text)["data"]):
            if user not in users and user != "":
                file.write(f"{user}\n")
                users.add(user)
        # page += 1

    revised_clubs.add(club_id)
    with open(f"{BASE_PATH}/_revised_clubs.txt", "a", encoding="UTF-8") as file:
        file.write(f"{club_id}\n")

41/41 --> 02

In [8]:
with open(f"{BASE_PATH}/users_list.txt", encoding="UTF-8") as file:
    users = list(set([x.strip() for x in file.readlines()]))[1:]
    random.shuffle(users)

with open(f"{BASE_PATH}/users.csv", "w", encoding="UTF-8") as file:
    file.write("user_id,username\n")
    for i, user in enumerate(users):
        file.write(f"{i},{user}\n")

# 3. Get animelist per user

In [9]:
with open(f"{BASE_PATH}/users.csv", "r", encoding="UTF-8") as file:
    file.readline()
    users = [x.strip().split(",") for x in file.readlines()]
    users = [(int(x[0]), x[1]) for x in users]

last_revised_users = -1
if os.path.exists(f"{BASE_PATH}/_last_revised_users.txt"):
    with open(f"{BASE_PATH}/_last_revised_users.txt", "r", encoding="UTF-8") as file:
        last_revised_users = int(file.readline())

len(users), last_revised_users

(1206, -1)

In [ ]:
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [11]:
#%%notify -m "animelist finish"
for i, (user_id, username) in enumerate(users):
    if user_id <= last_revised_users:
        continue

    now = datetime.now()
    print(f'\r{str(now).split(".")[0]} --> {i+1}/{len(users)}', end="")
    page = 1
    all_animes = []
    offset = 0
    while True:
    # link = f"https://api.jikan.moe/v4/user/{username}/animelist/all?page={page}"
        link = f"https://myanimelist.net/animelist/{username}/load.json?offset={offset}"
        try:
            time.sleep(4.2)
            data = requests.get(link, timeout=15)
        except KeyboardInterrupt:
            raise KeyboardInterrupt()
        except:  # Other exception wait 2 min and try again
            time.sleep(120)
            continue


        # if data.status_code != 200:
        #     break

        data = json.loads(data.text)

        for anime in data:
            try:
                all_animes.append((anime["anime_id"], anime["score"], anime["status"], anime["num_watched_episodes"]))
            except:
                print(username)

        offset += 300
        if len(data) < 300:
            break

    if len(all_animes) != 0:
        with open(f"{USER_PATH}/{user_id}.csv", "w") as f1:
            f1.write(f"anime_id,score,watching_status,watched_episodes\n")
            for anime_id, anime_score, watching_status, watched_episodes in all_animes:
                f1.write(
                    f"{anime_id},{anime_score},{watching_status},{watched_episodes}\n"
                )

    revised_users = user_id
    with open(f"{BASE_PATH}/_last_revised_users.txt", "w", encoding="UTF-8") as file:
        file.write(f"{user_id}\n")

2023-12-05 18:25:07 --> 7/1206-Aincrad-
2023-12-05 18:25:30 --> 10/1206Cieling_phan
2023-12-05 18:30:32 --> 38/1206-cutthroat954
2023-12-05 18:40:20 --> 89/1206-Luk134-
2023-12-05 18:40:33 --> 91/1206-Bass
2023-12-05 18:41:16 --> 95/1206-Ryuei
2023-12-05 18:41:21 --> 96/1206Disapeared_Ghost
2023-12-05 18:51:17 --> 127/12068arni
2023-12-05 18:53:16 --> 138/1206-LU-
2023-12-05 18:53:48 --> 140/1206DarknessAdmin
2023-12-05 18:55:22 --> 150/12067KpR3q2XeF9oWtPv
2023-12-05 18:57:09 --> 159/1206Blank0_0Zero
2023-12-05 19:02:05 --> 187/1206Amulet_Heart
2023-12-05 19:02:22 --> 190/12060moon0
2023-12-05 19:04:57 --> 205/1206Amelix
2023-12-05 19:08:30 --> 220/1206-chronos
2023-12-05 19:08:47 --> 222/1206Ame-Yume
2023-12-05 20:49:02 --> 301/12062Y
2023-12-05 20:53:36 --> 323/1206-Roxana
2023-12-05 20:57:53 --> 338/1206Beatrize
2023-12-05 20:59:04 --> 348/1206--Nahno--
2023-12-05 20:59:42 --> 352/1206170RheaRhea
2023-12-05 21:01:44 --> 363/1206-Sazamia-
2023-12-05 21:04:35 --> 376/1206-aejay-
2023

# 4. Download Anime HTML

In [3]:
unique_anime = set()
folder = os.listdir(USER_PATH)
for i, user_file in enumerate(folder):
    if ".csv" not in user_file:
        continue

    print(f"\r{i + 1}/{len(folder)}", end="")
    with open(f"{USER_PATH}/{user_file}", "r") as file:
        file.readline()
        for line in file:
            anime = line.strip().split(",")[0]
            unique_anime.add(anime)

print("         ")
print(len(unique_anime))

1094/1094         
19632


In [5]:
MAX = 7  # MAX SECOND TO WAIT PER REQUEST
MIN = 4  # MIN SECONDS TO WAIT PER REQUEST


def sleep():
    time_to_sleep = random.random() * (MAX - MIN) + MIN
    time.sleep(time_to_sleep)


def get_link_by_text(soup, anime_id, text):
    links = list(filter(lambda x: anime_id in x["href"], soup.find_all("a", text=text)))
    return links[0]["href"]


def save(path, data):
    with open(path, "w", encoding="UTF-8") as file:
        file.write(data)


def save_link(link, anime_id, name):
    sleep()
    path = f"{HTML_PATH}/{anime_id}/{name}.html"
    data = requests.get(link)
    soup = BeautifulSoup(data.text, "html.parser")
    soup.script.decompose()
    save(path, soup.prettify())
    return soup


def save_reviews(link, anime_id):
    page = 1
    while True:
        sleep()
        actual_link = f"{link}?p={page}"
        data = requests.get(actual_link)
        soup = BeautifulSoup(data.text, "html.parser")
        reviews = soup.find_all("a", text="Overall Rating")
        if len(reviews) == 0:
            break

        path = f"{HTML_PATH}/{anime_id}/reviews_{page}.html"
        soup.script.decompose()
        save(path, soup.prettify())
        page += 1


def scrap_anime(anime_id):
    path = f"{HTML_PATH}/{anime_id}"
    os.makedirs(path, exist_ok=True)
    sleep()
    data = requests.get(f"https://myanimelist.net/anime/{anime_id}")

    anime_info = data.text
    soup = BeautifulSoup(anime_info, "html.parser")
    soup.script.decompose()
    save(f"{HTML_PATH}/{anime_id}/details.html", soup.prettify())

    link_review = get_link_by_text(soup, anime_id, "Reviews")
    link_recomendations = get_link_by_text(soup, anime_id, "Recommendations")
    link_stats = get_link_by_text(soup, anime_id, "Stats")
    link_staff = get_link_by_text(soup, anime_id, "Characters & Staff")
    link_pictures = get_link_by_text(soup, anime_id, "Pictures")

    save_link(link_pictures, anime_id, "pictures")
    save_link(link_staff, anime_id, "staff")
    save_link(link_stats, anime_id, "stats")
    save_link(link_recomendations, anime_id, "recomendations")
    save_reviews(link_review, anime_id)


def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(
                os.path.join(root, file),
                os.path.relpath(os.path.join(root, file), path),
            )


In [ ]:
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [6]:
#%%notify -m "Anime scrapping finish"

for i, anime_id in enumerate(unique_anime):
    if os.path.isfile(f"{HTML_PATH}/{anime_id}.zip"):
        continue

    print(f"\r{i+1}/{len(unique_anime)}", end="")

    try:
        scrap_anime(anime_id)
    except KeyboardInterrupt:
        break
    except:  # Other exception wait 2 min and try again
        time.sleep(120)
        continue

    path = f"{HTML_PATH}/{anime_id}"
    zipf = zipfile.ZipFile(f"{path}.zip", "w", zipfile.ZIP_DEFLATED)
    zipdir(path, zipf)
    zipf.close()

    shutil.rmtree(path)

1/19632

C:\Users\X1G6\AppData\Local\Temp\ipykernel_21068\1764674217.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  links = list(filter(lambda x: anime_id in x["href"], soup.find_all("a", text=text)))
C:\Users\X1G6\AppData\Local\Temp\ipykernel_21068\1764674217.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  reviews = soup.find_all("a", text="Overall Rating")


41/19632

# 5. Scrapping Anime info from HTML

In [7]:
def extract_zip(input_zip):
    input_zip = ZipFile(input_zip)
    return {name: input_zip.read(name) for name in input_zip.namelist()}

KEYS = ['MAL_ID', 'Name', 'Score', 'Genders', 'English name', 'Japanese name', 'Type', 'Episodes',
        'Aired', 'Premiered', 'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
        'Ranked', 'Popularity', 'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
        'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6', 'Score-5', 'Score-4',
        'Score-3', 'Score-2', 'Score-1']

In [27]:
def get_name(info):
    try:
        return info.find("h1", {"class": "title-name h1_bold_none"}).text.strip()
    except: 
        return ""

def get_english_name(info):
    try:
        span = info.findAll("span", {"class": "dark_text"})
        return span.parent.text.strip()
    except:
        return ""

def get_table(a_soup):
    try:
        return a_soup.find("div", {"class": "po-r js-statistics-info di-ib"})
    except:
        return ""

def get_score(stats):
    try:
        score = stats.find("span", {"itemprop": "ratingValue"})
        if score is None:
            return "Unknown"
        return score.text.strip()
    except:
        return ""

def get_gender(sum_info):
    try:
        text = ", ".join(
            [x.text.strip() for x in sum_info.findAll("span", {"itemprop": "genre"})]
        )
        return text
    except:
        return ""

def get_description(sum_info):
    try:
        return sum_info.find("td", {"class": "borderClass", "width": "225"})
    except:
        return ""

def get_all_stats(soup):
    try:
        return soup.find("div", {"id": "horiznav_nav"}).parent.findAll(
            "div", {"class": "spaceit_pad"}
        )
    except:
        return ""

def get_info_anime(anime_id):
    data = extract_zip(f"craw/html/{anime_id}.zip")
    anime_info = data["stats.html"].decode()
    soup = BeautifulSoup(anime_info, "html.parser")

    stats = get_table(soup)
    description = get_description(soup)
    anime_info = {key: "Unknown" for key in KEYS}

    anime_info["MAL_ID"] = anime_id
    anime_info["Name"] = get_name(soup)
    anime_info["Score"] = get_score(stats)
    anime_info["Genders"] = get_gender(description)

    for d in description.findAll("span", {"class": "dark_text"}):
        information = [x.strip().replace(" ", " ") for x in d.parent.text.split(":")]
        category, value = information[0], ":".join(information[1:])
        value.replace("\t", "")

        if category in ["Broadcast", "Synonyms", "Genres", "Score", "Status"]:
            continue

        if category in ["Ranked"]:
            value = value.split("\n")[0]
        if category in ["Producers", "Licensors", "Studios"]:
            value = ", ".join([x.strip() for x in value.split(",")])
        if category in ["Ranked", "Popularity"]:
            value = value.replace("#", "")
        if category in ["Members", "Favorites"]:
            value = value.replace(",", "")
        if category in ["English", "Japanese"]:
            category += " name"

        anime_info[category] = value

    # Stats (Watching, Completed, On-Hold, Dropped, Plan to Watch)
    for d in get_all_stats(soup)[:5]:
        category, value = [x.strip().replace(" ", " ") for x in d.text.split(":")]
        value = value.replace(",", "")
        anime_info[category] = value

    # Stast votes per score
    for d in get_all_stats(soup)[6:]:
        score = d.parent.parent.find("td", {"class": "score-label"}).text.strip()
        value = [x.strip().replace(" ", " ") for x in d.text.split("%")][1].strip(
            "(votes)"
        )
        label = f"Score-{score}"
        anime_info[label] = value.strip()

    for key, value in anime_info.items():
        if str(value) in ["?", "None found, add some", "None", "N/A", "Not available"]:
            anime_info[key] = "Unknown"
    return anime_info

In [29]:
get_info_anime(1754)

{'MAL_ID': 1754,
 'Name': 'Di Gi Charat Summer Special 2000',
 'Score': '',
 'Genders': 'Comedy',
 'English name': 'Unknown',
 'Japanese name': 'Di Gi Charat サマースペシャル2000',
 'Type': 'Special',
 'Episodes': '4',
 'Aired': 'Aug 22, 2000 to Aug 23, 2000',
 'Premiered': 'Unknown',
 'Producers': 'TBS',
 'Licensors': 'Sentai Filmworks',
 'Studios': 'Madhouse',
 'Source': 'Original',
 'Duration': '11 min. per ep.',
 'Rating': 'G - All Ages',
 'Ranked': '6615\r',
 'Popularity': '9047',
 'Members': '3787',
 'Favorites': '3',
 'Watching': '106',
 'Completed': '2746',
 'On-Hold': '57',
 'Dropped': '110',
 'Plan to Watch': '768',
 'Score-10': '107',
 'Score-9': '100',
 'Score-8': '254',
 'Score-7': '514',
 'Score-6': '421',
 'Score-5': '235',
 'Score-4': '105',
 'Score-3': '47',
 'Score-2': '16',
 'Score-1': '23',
 'Genre': 'Comedy\r\n           \n\r\n            Comedy'}

In [30]:
# Generate anime.tsvdf.to_csv(f"{BASE_PATH}/anime.tsv", index=False, sep="\t", encoding="UTF-8")

anime_revised = set()
exist_file = os.path.exists(f"{BASE_PATH}/anime.tsv")
actual_data = pd.DataFrame()
if exist_file:
    # If the file exist, include new data.
    actual_data = pd.read_csv(f"{BASE_PATH}/anime.tsv", sep="\t")
    anime_revised = list(actual_data.MAL_ID.unique())

actual_data.head()
total_data = []
zips = os.listdir(HTML_PATH)
for i, anime in enumerate(zips):
    if not ".zip" in anime:
        continue

    anime_id = int(anime.strip(".zip"))

    if int(anime_id) in anime_revised:
        continue

    print(f"\r{i+1}/{len(zips)} ({anime_id})", end="")

    anime_id = anime.strip(".zip")
    total_data.append(get_info_anime(anime_id))

if len(total_data):
    df = pd.DataFrame.from_dict(total_data)
    df["MAL_ID"] = pd.to_numeric(df["MAL_ID"])
    df = df.sort_values(by="MAL_ID").reset_index(drop=True)

    if exist_file:
        df = (
            pd.concat([actual_data, df]).sort_values(by="MAL_ID").reset_index(drop=True)
        )

else:
    df = actual_data

pd.set_option("display.max_columns", None)
df.head()

41/41 (9782))

,MAL_ID,Name,Score,Genders,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,French,Theme,Demographic,German,Spanish,Genre,Themes
0,55,Arc the Lad,,"Action, Adventure, Fantasy, Horror, Sci-Fi",Unknown,アークザラッド,TV,26,"Apr 5, 1999 to Oct 11, 1999",Spring 1999,"Sony Pictures Entertainment, Aniplex, Sony Int...",ADV Films,Bee Train,Game,22 min. per ep.,R - 17+ (violence & profanity),6610\r,5131,16308,16,745,7934,781,1036,5812,231,293,813,1734,1415,919,335,123,56,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,281,Akane Maniax,,"Comedy, Romance, Mecha",Unknown,アカネマニアックス,OVA,3,"Nov 25, 2004 to Aug 26, 2005",Unknown,"Lantis, Age",Unknown,"Media Factory, Silver",Visual novel,27 min. per ep.,PG-13 - Teens 13 or older,10491\r,4844,18954,9,438,12924,384,704,4504,254,333,883,1885,2113,1870,1049,544,337,240,NaN,Mecha\r\n \n\r\n Mecha,NaN,NaN,NaN,NaN,NaN
2,571,Moldiver,,"Action, Romance, Sci-Fi, Mecha",Unknown,モルダイバー,OVA,6,"Feb 25, 1993 to Nov 25, 1993",Unknown,Pioneer LDC,Geneon Entertainment USA,AIC,Original,29 min. per ep.,PG-13 - Teens 13 or older,7463\r,8760,4230,10,208,1880,131,222,1789,128,119,121,307,330,243,129,53,28,29,NaN,Mecha\r\n \n\r\n Mecha,NaN,NaN,NaN,NaN,NaN
3,828,Jinshin Yuugi,,Hentai,Love Lessons,人心遊戯,OVA,2,"Mar 21, 2001 to Jun 21, 2001",Unknown,Milky Animation Label,Central Park Media,Office AO,Visual novel,28 min. per ep.,Rx - Hentai,N/A\r,10584,2281,1,181,1149,103,139,709,47,20,66,110,162,175,67,54,22,40,NaN,NaN,NaN,NaN,NaN,Hentai\r\n \n\r\n Hentai,NaN
4,1023,Wolf's Rain OVA,,"Adventure, Drama, Sci-Fi, Supernatural, Psycho...",Wolf's Rain OVA,ウルフズ・レイン,OVA,4,"Jan 23, 2004 to Feb 25, 2004",Unknown,"Bandai Visual, Asatsu DK","Funimation, Bandai Entertainment",Bones,Original,23 min. per ep.,PG-13 - Teens 13 or older,622\r,2577,67095,373,1252,46766,828,553,17696,6340,7357,9525,6444,2743,1244,464,220,118,93,Wolf's Rain,Psychological\r\n \n\r\n ...,NaN,NaN,NaN,NaN,NaN


In [31]:
df.to_csv(f"{BASE_PATH}/anime.tsv", index=False, sep="\t", encoding="UTF-8")


# 6. Create rating_complete.csv

This file only contain animes with `watching_status==2`(complete) and have been rated (`score!=0`).

In [32]:
if not os.path.exists(f"{BASE_PATH}/rating_complete.csv"):
    with open(f"{BASE_PATH}/rating_complete.csv", "w", encoding="UTF-8") as file:
        file.write("user_id,anime_id,rating\n")

In [33]:
unique_anime = set()
all_users = sorted(os.listdir(USER_PATH), key=lambda x:int(x.split(".")[0]))

with open(f"{BASE_PATH}/rating_complete.csv", "a") as f1:

    for i, user_file in enumerate(all_users):
        if not user_file.endswith(".csv"):
            continue

        print(f"\r{i+1}/{len(all_users)}", end="")

        user_id = user_file.split(".")[0]
        with open(f"{USER_PATH}/{user_file}", "r") as file:
            file.readline()
            for line in file:
                anime_id, score, watching_status, _ = line.strip().split(",")
                if int(watching_status) == 2 and (score) != 0:
                    f1.write(f"{user_id},{anime_id},{score}\n")

1094/1094

# 7. Unified animelist

In [ ]:
if not os.path.exists(f"{BASE_PATH}/animelist.csv"):
    with open(f"{BASE_PATH}/animelist.csv", "w", encoding="UTF-8") as file:
        file.write("user_id,anime_id,rating,watching_status,watched_episodes\n")

In [34]:
unique_anime = set()
all_users = sorted(os.listdir(USER_PATH), key=lambda x:int(x.split(".")[0]))

with open(f"{BASE_PATH}/animelist.csv", "a") as f1:

    for i, user_file in enumerate(all_users):
        if not user_file.endswith(".csv"):
            continue

        print(f"\r{i+1}/{len(all_users)}", end="")

        user_id = user_file.split(".")[0]
        with open(f"{USER_PATH}/{user_file}", "r") as file:
            file.readline()
            for line in file:
                anime_id, score, watching_status, watched_episodes = line.strip().split(",")
                f1.write(f"{user_id},{anime_id},{score},{watching_status},{watched_episodes}\n")

1094/1094